# Main document for assignment


In [1]:
from DataGeneration import DataGeneration

In [4]:
tmp = DataGeneration(2016)
tmp.data

,cet_cest_timestamp,DE_load_actual_entsoe_transparency,DE_solar_profile,DE_wind_profile,DE_wind_offshore_profile,DK_load_actual_entsoe_transparency,DK_1_load_actual_entsoe_transparency,DK_2_load_actual_entsoe_transparency,NO_load_actual_entsoe_transparency
utc_timestamp,,,,,,,,,
2016-01-01 00:00:00+00:00,2016-01-01T01:00:00+0100,40126.0,0.0,0.2614,1.0571,3028.24,1740.37,1287.87,15128.76
2016-01-01 01:00:00+00:00,2016-01-01T02:00:00+0100,38429.0,0.0,0.2603,1.0309,2888.52,1657.54,1230.98,14959.02
2016-01-01 02:00:00+00:00,2016-01-01T03:00:00+0100,37484.0,0.0,0.2573,0.9304,2761.57,1592.20,1169.37,14706.77
2016-01-01 03:00:00+00:00,2016-01-01T04:00:00+0100,37150.0,0.0,0.2528,0.8986,2689.48,1547.87,1141.61,14662.90
2016-01-01 04:00:00+00:00,2016-01-01T05:00:00+0100,36388.0,0.0,0.2231,0.7320,2695.61,1552.15,1143.46,14879.68
...,...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00+00:00,2016-12-31T20:00:00+0100,49558.0,0.0,0.4120,1.2796,3236.85,1785.64,1451.21,15771.48
2016-12-31 20:00:00+00:00,2016-12-31T21:00:00+0100,47558.0,0.0,0.4134,1.2771,3071.10,1696.56,1374.54,15483.58
2016-12-31 21:00:00+00:00,2016-12-31T22:00:00+0100,47600.0,0.0,0.3997,1.2780,2945.06,1630.62,1314.44,15237.38


# Task A

In [ ]:
import pypsa
import pandas as pd

technology_data = {'gas': {'Marginal_cost': 21.6, 'Efficiency': 0.39, 'Capital_cost': 560000, 'Lifetime': 25, 'CO2_emissions': 0.19},
                   'onshorewind': {'Marginal_cost': 21.6, 'Efficiency': 0.39, 'Capital_cost': 560000, 'Lifetime': 25, 'CO2_emissions': 0.19},
                   'Solar': {'Marginal_cost': 21.6, 'Efficiency': 0.39, 'Capital_cost': 560000, 'Lifetime': 25, 'CO2_emissions': 0.19},
             }

class BuildBaseNetwork:
    def __init__(self, year: int = 2019, profile_data = pd.DataFrame, technology_data = dict, interest_rate = 0.07):
        
        self.year = year

        self.profile_data = profile_data
        self.technology_data = technology_data
        self.interest_rate = interest_rate

        self.network = pypsa.Network()
        self.hours_in_year = pd.date_range(f'{year}-01-01 00:00', f'{year}-12-31 23:00', freq='h')
        self.network.set_snapshots(self.hours_in_year.values)

        self.add_busses()

        self.network.optimize(solver_name='gurobi')

    def add_busses(self):
        self.network.add("Bus", 'electricity bus')
        self.network.add("Load", 'load', bus = 'electricity bus', p_set = self.profile_data['DK_load_actual_entsoe_transparency'].values)
        # add the different carriers, only gas emits CO2


        # self.network.add("Carrier", "gas", co2_emissions=0.19) # in t_CO2/MWh_th
        self.network.add("Carrier", "gas", co2_emissions = self.technology_data['gas']['CO2_emissions'])
        # add OCGT (Open Cycle Gas Turbine) generator
        capital_cost_OCGT = self.annuity(25, self.interest_rate)*560000*(1+0.033) # in €/MW
        fuel_cost = 21.6 # in €/MWh_th
        efficiency = 0.39 # MWh_elec/MWh_th
        marginal_cost_OCGT = fuel_cost/efficiency # in €/MWh_el
        
        self.network.add("Generator",
                    "OCGT",
                    bus="electricity bus",
                    p_nom_extendable=True,
                    carrier="gas",
                    #p_nom_max=1000,
                    capital_cost = capital_cost_OCGT,
                    marginal_cost = marginal_cost_OCGT)

        self.network.add("Carrier", "onshorewind")
        # add onshore wind generator
        CF_wind = self.profile_data['DK_wind_onshore_generation_actual'].values
        capital_cost_onshorewind = self.annuity(30, self.interest_rate) * self.technology_data['onshorewind']['Capital_cost'] * (1+0.033) # in €/MW
        self.network.add("Generator",
                    "onshorewind",
                    bus="electricity bus",
                    p_nom_extendable=True,
                    carrier="onshorewind",
                    #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
                    capital_cost = capital_cost_onshorewind,
                    marginal_cost = 0,
                    p_max_pu = CF_wind)

        self.network.add("Carrier", "solar")

        # add solar PV generator
        CF_solar = self.profile_data['DK_solar_generation_actual'].values
        capital_cost_solar = self.annuity(25,0.07)*425000*(1+0.03) # in €/MW
        self.network.add("Generator",
                    "solar",
                    bus="electricity bus",
                    p_nom_extendable=True,
                    carrier="solar",
                    #p_nom_max=1000, # maximum capacity can be limited due to environmental constraints
                    capital_cost = capital_cost_solar,
                    marginal_cost = 0,
                    p_max_pu = CF_solar)

    def add_line(self, name, bus0, bus1, length, s_nom):
        self.network.add("Line", name, bus0, bus1, length=length, s_nom=s_nom)

    @staticmethod
    def annuity(n, r):
        """ Calculate the annuity factor for an asset with lifetime n years and
        discount rate  r """

        if r > 0:
            return r/(1. - 1./(1.+r)**n)
        else:
            return 1/n


In [7]:
technology_data['onshorewind']['Capital_cost']



560000